In [4]:
from nipype import SelectFiles, Node, MapNode, Workflow, Function
from nipype.interfaces import fsl, ants
from os.path import abspath
from IPython.display import Image
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import ImageStats
from nipype import config

cfg = dict(execution={'remove_unnecessary_outputs': False,
                      'keep_inputs': True}
          )
config.update_config(cfg)

In [ ]:
# Node: SelectFiles
## Selects input file "test_1.nii.gz" from directory "/Users/fgiuste/Google Drive/Luyuan_Gliomics"
# test_1.nii.gz: 100x200x450x30
templates={'pre': 'TCGA-*/[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]/T1-AXIAL_PREGD.nii.gz'
          }
sfT1 = Node(SelectFiles(templates),
          name='selectT1'
          )
sfT1.inputs.base_directory = '/data/'

#overwrite = True
sfT1.run().outputs



In [ ]:
# Run FSL FLIRT

flrt = MapNode(fsl.FLIRT(bins=640),
               name = 'FLIRT',
               iterfield = 'in_file',
               overwrite = False
              )

flt.inputs.reference = '/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm.nii.gz'
# MNI152_T1_1mm_brain.nii.gz
# MNI152_T1_1mm_brain_mask.nii.gz
flrt.inputs.output_type = "NIFTI_GZ"


In [ ]:
# Run FSL FNRT

fnrt = MapNode(fsl.FNIRT(),
               name = 'FNIRT',
               iterfield = 'in_file',
               overwrite = False
              )
fnlt.inputs.reference = 'mni152.nii.gz'

In [ ]:
# Run FSL ANTS

ants = MapNode(ants.ANTS(),
               name = 'ANTS',
               iterfield = 'in_file',
               overwrite = False
              )

In [ ]:
# Workflow: Initialize
wf = Workflow(name="MRI_Registration")
wf.base_dir = sfT1.inputs.base_directory

In [ ]:
# Make Connections

wf.connect(sfT1, "pre", flrt, "in_file")
wf.connect(sfT1, "pre", fnrt, "in_file")
wf.connect(sfT1, "pre", ants, "in_file")

wf.connect(flrt, "affine_file", fnrt, "affine_file")


In [ ]:
# Workflow: Graph: Exec
wf.write_graph(graph2use='exec', dotfilename= sfT1.inputs.base_directory+'/graph_exec.dot' )

# Visualize graph
Image(filename= sfT1.inputs.base_directory+'/graph_exec.png' )

In [ ]:
wf.run(plugin='MultiProc', plugin_args={'n_procs': 8})
# wf.run()